## Многокритериальная задача
#### Козловский А.А., гр. 2253

In [2]:
import numpy as np
import pandas as pd
import copy

In [3]:
def matrix_by_rows(matrix, right_rows):
    """
    Функция для построение новой матрицы по строкам
    
    matrix: изначальная матрица [np.array]
    right_rows: строки, которые надо включить в новую матрицу [np.array]
    
    returns: новую матрицу [np.array]
    """
    
    result = []
    for row in right_rows:
        if row != -1:
            result.append(matrix[row])
            
    result = np.array(result)
    return result

In [4]:
def compare_pareto(matrix, row_1, row_2, features, types_of_features):
    """
    Метод для определения доминирования одной альтернативы над другой по Парето
    
    matrix: матрица признаков
    row_1: 1 строка матрицы признаков
    row_2: 2 строка матрицы признаков
    features: рассматриваемые критерии [np.array]
    types_of_features: виды критериев (-1 - негативный, 1 - позитивные) [np.array]
    
    returns: True - если первая строка доминирует вторую по Парето, иначе - False 
    """
    
    result = False
    for i, feature in enumerate(features):
        if types_of_features[i] * row_1[feature] < types_of_features[i] * row_2[feature]:
            return False
        if row_1[feature] != row_2[feature]:
            result = True
            
    return result

In [5]:
def make_pareto_optimal(matrix, features, types_of_features):
    """
    Функция построение Парето-оптимального множества
    
    matrix: матрица признаков [np.array]
    features: рассматриваемые критерии [np.array]
    types_of_features: виды критериев (-1 - негативный, 1 - позитивные) [np.array]
    
    returns: новую матрицу признаков для оптимального множетсва [np.array]
    """
    
    right_rows = np.arange(0, matrix.shape[0], 1, dtype=int)
    for index_1, row_1 in enumerate(matrix):
        for index_2, row_2 in enumerate(matrix):
            if index_1 == index_2:
                continue
            if compare_pareto(matrix, row_1, row_2, features, types_of_features):
                right_rows[index_2] = -1
         
    return matrix_by_rows(matrix, right_rows)

In [6]:
def low_bounds_method(matrix, features, types_of_features, low_bounds):
    """
    Функция для сужения оптимального множества методом нижних границ
    
    matrix: матрица признаков [np.array]
    features: рассматриваемые критерии [np.array]
    types_of_features: виды критериев (-1 - негативный, 1 - позитивные) [np.array]
    low_bounds: нижние границы для соответствующих критерие [np.array]
    
    returns: новую матрицу признаков для оптимального множетсва [np.array]
    """
    
    matrix_pareto = make_pareto_optimal(matrix, features, types_of_features)
    right_rows = np.arange(0, matrix_pareto.shape[0], 1, dtype=int)
    
    for row_num, row in enumerate(matrix_pareto):
        for feature_num, feature in enumerate(features):
            if (types_of_features[feature_num] * row[feature] < 
                types_of_features[feature_num] * low_bounds[feature_num]):
                right_rows[row_num] = -1
                break
                   
    return matrix_by_rows(matrix_pareto, right_rows)

In [76]:
def suboptimization(matrix, features, types_of_features, low_bounds, main_feature):
    """
    Функция для нахождения оптимального множества альтернатив с помощью
    субоптимизации
    
    matrix: матрица признаков [np.array]
    features: рассматриваемые критерии [np.array]
    types_of_features: виды критериев (-1 - негативный, 1 - позитивные) [np.array]
    low_bounds: нижние границы для соответствующих критерие [np.array]
    main_feature: главный критерий
    
    returns: новую матрицу признаков для оптимального множетсва [np.array]
    """
    
        
    
    right_rows = np.arange(0, matrix.shape[0], 1, dtype=int)
    for row_num, row in enumerate(matrix):
        for feature_num, feature in enumerate(features):
            if feature_num == main_feature:
                continue
            if (types_of_features[feature_num] * row[feature] < 
                types_of_features[feature_num] * low_bounds[feature_num]):
                right_rows[row_num] = -1
                break
             
    main_feature_matrix = matrix_by_rows(matrix, right_rows)
    right_rows = np.arange(0, main_feature_matrix.shape[0], 1, dtype=int)
    
    max_value = (types_of_features[main_feature] * 
        np.max(main_feature_matrix.T[features[main_feature]] * types_of_features[main_feature]))
    
    result = []
    for i, row in enumerate(main_feature_matrix):
        if row[features[main_feature]] == max_value:
            result.append(row)

    return result

In [8]:
def lexicographical_optimization(matrix, features, types_of_features):
    """
    Функция для лексикографической оптимизации
    
    matrix: матрица признаков [np.array]
    features: рассматриваемые критерии [np.array]
    types_of_features: виды критериев (-1 - негативный, 1 - позитивные) [np.array]
    
    returns: новую матрицу признаков для оптимального множетсва [np.array]
    """
    
    matrix_copy = matrix.copy()
    for feature_num, feature in enumerate(features):
        result = []
        max_value = (types_of_features[feature_num] * 
                    np.max(matrix_copy.T[feature] * types_of_features[feature_num]))
        for row in matrix_copy:
            if row[feature] == max_value:
                result.append(row)
                
        matrix_copy = np.array(result)
        if matrix_copy.shape[0] == 1:
            break
            
    return result

In [9]:
def general_feature_optimization(matrix, features, types_of_features, weights):
    """
    Функция для построения оптимального множества по обобщенному критерию
    
    matrix: матрица признаков [np.array]
    features: рассматриваемые критерии [np.array]
    types_of_features: виды критериев (-1 - негативный, 1 - позитивные) [np.array]
    weights: веса для построения обощенного критерия [np.array]
    
    returns: новую матрицу признаков для оптимального множетсва [np.array]
    """
    
    matrix_copy = matrix.copy()
    # Нормализуем значения признаков
    for feature in features:
        matrix_copy.T[feature] /= np.max(matrix_copy.T[feature] - np.min(matrix_copy.T[feature]))
        
    general_features = np.zeros(matrix_copy.shape[0])
    for index, general_feature in enumerate(general_features):
        for feature_num, feature in enumerate(features):
            general_features[index] += (matrix_copy[index][feature] * 
                               weights[feature_num] * types_of_features[feature_num]) 
            
    right_rows = np.argwhere(general_features == np.max(general_features))
    result = np.zeros((right_rows.shape[0], matrix_copy.shape[1]), dtype=object)
    for index, result_elemnt in enumerate(result):
        result[index] = matrix[right_rows[index]]
        
    return result

### Решение многокритериальной задачи
#### Подготовка данных
Подготовим данные для решения задачи. Для этого возьмем датасет с книгами, подготовленный в предыдущем задании.
Обозначим три критерия, по которым будет решаться многокритериальная задача. В нашем случае это будут: цена, рейтинг, количество оценок.

In [10]:
data = pd.read_csv('full_dataset3.csv')
matrix = data.to_numpy()
features = [-1, -2, -3]

### Формирование множества Парето
Сформируем множество Парето по трем выбранным критериям и датасету

In [95]:
matrix_pareto = make_pareto_optimal(matrix, features, [1, 1, -1])
pareto_df = pd.DataFrame(columns=data.columns, data=matrix_pareto)

In [96]:
pareto_df

,name,author,translator,editor,publisher,genre,series,articul,isbn,pages2,weight,buying-price-val-number,rate,count-marks-label
0,Пётр I,"Новичкова Елена, Ратина Анна, Бунтман Екатерина",NaN,Новичкова Елена,Лабиринт,NaN,Детская художественная литература,507700,978-5-17-134106-0,2820,1200,3680,7.8,327
1,Грокаем алгоритмы. Иллюстрированное пособие дл...,Бхаргава Адитья,Матвеев Е.,Римицан Н.,Питер,NaN,Библиотека программиста,571060,978-5-17-134106-0,288,376,1024,8.89,44
2,Компьютерные сети,"Таненбаум Эндрю, Уэзеролл Дэвид",Гребеньков А.,Сергиенко Ю.,Питер,NaN,Классика computer science,310025,978-5-17-134106-0,960,1344,2312,9.47,40
3,Идеальный программист. Как стать профессионало...,Мартин Роберт С.,Матвеев Е.,Сергиенко Ю.,Питер,NaN,NaN,643363,978-5-17-134106-0,224,300,845,10,10
4,Современные операционные системы,"Таненбаум Эндрю, Бос Херберт","Леонтьева А. В., Малышева М., Вильчинский Н.",Сергиенко Ю.,Питер,NaN,Классика computer science,485463,978-5-17-134106-0,1120,1550,2178,9.48,21
5,"Дизайн персонажей. Концепт-арт для комиксов, в...","Кэттиш Анна, Че Тата, Смирнов Иван",Семенова Д.,NaN,Питер,NaN,Компьютерная графика и мультимедиа,780884,978-5-17-134106-0,272,942,2837,9.56,16
6,"Смартфон для тех, кто ни бум-бум в телефонах",Левина Любовь Тимофеевна,NaN,NaN,АСТ,NaN,Энциклопедия ржавого чайника,778712,978-5-17-134106-0,208,248,304,0,0
7,"Планшет для тех, кто ни бум-бум в компьютерах",Левина Любовь Тимофеевна,NaN,Максимова О.,АСТ,NaN,Энциклопедия ржавого чайника,775374,978-5-17-134106-0,224,266,304,0,0
8,"Чистый код. Создание, анализ и рефакторинг",Мартин Роберт С.,Матвеев Е.,Сергиенко Ю.,Питер,NaN,NaN,642466,978-5-17-134106-0,464,600,921,9.07,14
9,Компьютерные сети,"Таненбаум Эндрю, Уэзеролл Дэвид",Гребеньков А.,Сергиенко Ю.,Питер,NaN,Классика computer science,310025,978-5-17-134106-0,960,1344,2312,9.47,40


### Указание нижних границ критериев
Воспользуемся вторым методом. Найдем множество Парето оптимальных альтернатив, а потом назначим нижние допустимые границы критериев, чтобы сформировать финальное множество

In [98]:
matrix_low_bound = low_bounds_method(matrix, features, [1, 1, -1], [10, 6.5, 2000])
low_bound_df = pd.DataFrame(columns=data.columns, data=matrix_low_bound)

In [99]:
low_bound_df

,name,author,translator,editor,publisher,genre,series,articul,isbn,pages2,weight,buying-price-val-number,rate,count-marks-label
0,Грокаем алгоритмы. Иллюстрированное пособие дл...,Бхаргава Адитья,Матвеев Е.,Римицан Н.,Питер,NaN,Библиотека программиста,571060,978-5-17-134106-0,288,376,1024,8.89,44
1,Идеальный программист. Как стать профессионало...,Мартин Роберт С.,Матвеев Е.,Сергиенко Ю.,Питер,NaN,NaN,643363,978-5-17-134106-0,224,300,845,10,10
2,"Чистый код. Создание, анализ и рефакторинг",Мартин Роберт С.,Матвеев Е.,Сергиенко Ю.,Питер,NaN,NaN,642466,978-5-17-134106-0,464,600,921,9.07,14
3,Procreate. Учимся создавать шедевры на Ipad. Е...,"Уличнэй Макс, Нассур Сэм, Стокарт Эвелин, Грюн...",Стеблева О.,Апалина А.,Бомбора,NaN,Учимся рисовать на компьютере и планшете,773508,978-5-17-134106-0,216,1108,1340,9.57,14
4,Идеальный программист. Как стать профессионало...,Мартин Роберт С.,Матвеев Е.,Сергиенко Ю.,Питер,NaN,NaN,643363,978-5-17-134106-0,224,300,845,10,10
5,Программируем на Python,Доусон Майкл,NaN,Гринчик Н.,Питер,NaN,NaN,311244,978-5-17-134106-0,416,536,1393,9.11,44
6,Все секреты Minecraft. Читы и командные блоки,Миллер Меган,Райтман М. А.,Иванова В.,Бомбора,Артбуки. Игровые миры. Вселенные,Minecraft,590450,978-5-17-134106-0,12870,340,450,8.5,18
7,Основы программирования на языке Python,Златопольский Дмитрий Михайлович,NaN,Мовчан Д. А.,ДМК-Пресс,NaN,NaN,668926,978-5-17-134106-0,396,390,827,8.56,16
8,"Чистый код. Создание, анализ и рефакторинг",Мартин Роберт С.,Матвеев Е.,Сергиенко Ю.,Питер,NaN,NaN,642466,978-5-17-134106-0,464,600,921,9.07,14
9,Procreate. Учимся создавать шедевры на Ipad. Е...,"Уличнэй Макс, Нассур Сэм, Стокарт Эвелин, Грюн...",Стеблева О.,Апалина А.,Бомбора,NaN,Учимся рисовать на компьютере и планшете,773508,978-5-17-134106-0,216,1108,1340,9.57,14


### Субоптимизация
Выделим множество альтернатив, удовлетворяющих по нужным критериям нижним границам, и найдем максимум по главному критерию

In [100]:
matrix_sub = suboptimization(matrix, [-1, -2, -3], [1, 1, -1], [20, 5, 2000], 2)
sub_df = pd.DataFrame(columns=data.columns, data=matrix_sub)

In [101]:
sub_df

,name,author,translator,editor,publisher,genre,series,articul,isbn,pages2,weight,buying-price-val-number,rate,count-marks-label
0,Статистика и котики,Савельев Владимир,NaN,Кольцова В.,АСТ,Машинное обучение. Анализ данных,Звезда Рунета. Бизнес,638687,978-5-17-134106-0,19240.0,324.0,598,8.27,26


### Лексикографическая оптимизация
Упорядочим критерии в порядке важности. Для этого просто укажем их в соответствующем порядке при передаче как параметр в нашу функцию. В порядке убывания важности наши критерии будут выглядеть так: рейтинг, цена, количество оценок

In [102]:
matrix_lex = lexicographical_optimization(matrix, [-2, -3, -1], [1, -1, 1])
lex_df = pd.DataFrame(columns=data.columns, data=matrix_lex)

In [103]:
lex_df

,name,author,translator,editor,publisher,genre,series,articul,isbn,pages2,weight,buying-price-val-number,rate,count-marks-label
0,Компьютер и Интернет с самых азов. Максимально...,Жуков Иван,NaN,NaN,АСТ,NaN,Современный самоучитель,758201,978-5-17-134106-0,352.0,342.0,385,10.0,1


### Построение обощенного критерия
Для построения обобщенного критерия будем использовать веса 0.1, 0.4, 0.5 для количества оценивших, рейтинга и цены соответственно.

In [104]:
matrix_general = general_feature_optimization(matrix, [-1, -2, -3], [1, 1, -1], [0.1, 0.4, 0.5])
general_df = pd.DataFrame(columns=data.columns, data=matrix_general)

In [105]:
general_df

,name,author,translator,editor,publisher,genre,series,articul,isbn,pages2,weight,buying-price-val-number,rate,count-marks-label
0,Компьютер и Интернет с самых азов. Максимально...,Жуков Иван,NaN,NaN,АСТ,NaN,Современный самоучитель,758201,978-5-17-134106-0,352,342,385,10,1


,name,author,translator,editor,publisher,genre,series,articul,isbn,pages2,weight,buying-price-val-number,rate,count-marks-label
0,Пётр I,"Новичкова Елена, Ратина Анна, Бунтман Екатерина",NaN,Новичкова Елена,Лабиринт,NaN,Детская художественная литература,507700,978-5-17-134106-0,2820.0,1200.0,3680,7.80,327
1,Unity и C#. Геймдев от идеи до реализации,Бонд Джереми Гибсон,Киселев А.,Тульцева К.,Питер,NaN,Для профессионалов,686355,978-5-17-134106-0,928.0,1190.0,2975,9.70,10
2,Грокаем алгоритмы. Иллюстрированное пособие дл...,Бхаргава Адитья,Матвеев Е.,Римицан Н.,Питер,NaN,Библиотека программиста,571060,978-5-17-134106-0,288.0,376.0,1024,8.89,44
3,Компьютерные сети,"Таненбаум Эндрю, Уэзеролл Дэвид",Гребеньков А.,Сергиенко Ю.,Питер,NaN,Классика computer science,310025,978-5-17-134106-0,960.0,1344.0,2312,9.47,40
4,Интерфейс. Основы проектирования взаимодействия,"Купер Алан, Носсел Кристофер, Кронин Дэвид, Ре...",Матвеев Е.,Римицан Н.,Питер,NaN,Для профессионалов,521205,978-5-17-134106-0,720.0,914.0,2197,9.15,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,Численные методы и программирование. Учебное п...,Слабнов Виктор Дмитриевич,NaN,NaN,Лань,NaN,Компьютеры и программное обеспечение,786067,978-5-17-134106-0,460.0,714.0,3975,0.00,0
260,Web-программирование на JavaScript. Учебное по...,Диков Андрей Валентинович,NaN,NaN,Лань,NaN,Компьютеры и программное обеспечение,786072,978-5-17-134106-0,168.0,278.0,1492,0.00,0
261,Технология разработки программного обеспечения...,Зубкова Татьяна Михайловна,NaN,NaN,Лань,NaN,Компьютеры и программное обеспечение,786076,978-5-17-134106-0,252.0,450.0,2386,0.00,0
262,Основы построения инфокоммуникационных сетей и...,"Пуговкин Алексей Викторович, Покаместов Дмитри...",NaN,NaN,Лань,NaN,Компьютеры и программное обеспечение,786078,978-5-17-134106-0,176.0,354.0,1889,1.00,1
